In [1]:
using WordNet
using AdaGram
using AdaGramCompat

using CorpusLoaders
using CorpusLoaders.Semcor
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using LightXML
using JLD

In [2]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [3]:
#am = AdaGramCompat.AdaGramModel(load_model("models/adagram/v1_d100.adagram_model")...)
#am = AdaGramModel(load_model("models/adagram/more_senses.adagram_model")...)
#am = open(deserialize,"models/adagram/more_senses.adagram_model.jsz", "r");
am = load("models/adagram/more_senses.adagram_model.jld", "am");


In [167]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld");
ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld", "ee")

#ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

In [9]:
reload("CorpusLoaders")

In [10]:
using CorpusLoaders.Semeval2007t7

challenges = load_challenges_semeval2007t7("data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml");
solutions = load_solutions_semeval2007t7("data/corpora/wsd/semeval2007_t7/key/dataset21.test.key");

In [11]:
semcor = index_semcor(lazyload_semcor("data/corpora/semcor2.1/brown1/tagfiles/"));

In [12]:
only_of_pos(data, pos) = filter(d->d.pos==pos, data)

only_of_pos (generic function with 1 method)

In [13]:
function sense_frequency(ss::Synset, lem::Lemma)
    lem_count = lem.tagsense_count
    #lem_count+0.1(sum(values(ss.word_counts))-lem_count) #Do not use Synset's other counts. It owrks out worse
    lem_count
end

sense_frequency (generic function with 1 method)

In [15]:
#typealias Usages{S<:AbstractString, V<:AbstractVector{S}} Dict{Synset, Abs }

In [112]:
"Collect up the usages from a indexed tagged source"
function get_usages(usage_index::Dict{String, Vector{Semcor.TaggedSentence}}, key)
    if haskey(usage_index, key)
        [map(lowercase, strip_tags(sent)) for sent in usage_index[key]]
    else
        Vector{String}[]
    end
end

function get_usages(synset::Synset)
    gloss::Vector{SubString{String}} = map(lowercase, punctuation_space_tokenize(synset.gloss))
    [gloss]
end

function get_all_usages(wn::DB, semcor::Dict{String, Vector{Semcor.TaggedSentence}}, lemma_word, pos)   
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    
    Dict{Synset,AbstractVector{AbstractVector}}((synset => get_usages(synset)
        #get_usages(semcor, sensekey(db, synset, lemma)); get_usages(synset)]
                    for synset in target_synsets))  
    
end

get_all_usages (generic function with 1 method)

In [113]:
function all_word_sense_vectors(ee::WordSenseEmbedding, word)
    get(ee.embedding, word, Vector{Float32}[])
end

function all_word_sense_vectors(am::AdaGramCompat.AdaGramModel, word)
    if haskey(am.dict.word2id, word)
        wsv_mat = word_sense_vectors(am, word)
        [view(wsv_mat,:,ii) for ii in 1:size(wsv_mat,2)]
    else
        Vector{Float32}[]
    end
end

all_word_sense_vectors (generic function with 2 methods)

In [131]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    @show mean(logprobs)
    @show max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    @show ret
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end


function synthesize_embedding1(ee,context::AbstractVector, word_or_phrase::AbstractString)
    words = split(word_or_phrase, " ")
    wvs = vcat((all_word_sense_vectors(ee,w) for w in words)...)
    if length(wvs) == 0
            throw(KeyError("None of $words have embeddings"))
    end
        logprobs = [Query.logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=false) for wv in wvs]
    weighted_average(logprobs, wvs)
end

    

function synthesize_embedding(am::AdaGramModel,context::AbstractVector, word::AbstractString)
    known_context = filter(c->haskey(am.dict.word2id, c), context)
    sum(all_word_sense_vectors(am, word).*disambiguate(am.vm, am.dict, word, known_context))
end

synthesize_embedding (generic function with 2 methods)

###Note the disambig prob fumctions :
 - Query.logprob_of_context(am, context, wv; , normalise_over_length=false)
- Adagram.disambiguate(am.vm, am.dict, word, known_context, false))
Are identical. That is with normalise_over_length false for Logprob-of-context, 
and use_prior false for Adagram.disambiguate 

In [132]:
w = "bank"
uses = get_all_usages(db, semcor, w, 'n')
l_uses= uses[collect(keys(uses))[3]]
l_use = l_uses[1]
#l_use = split("i swam in the water near the i saw a fish")
@show l_use
wv = synthesize_embedding1(am,l_use, w)
nearest_neighbors(am.vm, am.dict, wv)

l_use = SubString{String}["sloping","land","especially","the","slope","beside","a","body","of","water","they","pulled","the","canoe","up","on","the","bank","he","sat","on","the","bank","of","the","river","and","watched","the","currents"]
mean(logprobs) = -223.88046f0
max_lp = maximum(logprobs) = -209.24814f0
ret = Float32[2.74554f-16,1.33641f-19,1.94756f-11,9.23107f-19,1.13482f-13,2.64748f-22,1.55055f-11,2.565f-21,8.18769f-21,2.6063f-16,1.66657f-14,9.36849f-12,3.15721f-16,0.0441032,0.0618453,0.0592532,0.0649737,0.059418,0.0608503,0.0634502,0.0580682,0.0578868,0.0635035,0.0656825,0.0569155,0.0518538,0.0572447,0.0552748,0.0535602,0.0661159]


10-element Array{Tuple{AbstractString,Int64,Float32},1}:
 ("doms",2,0.00349899)      
 ("identifed",2,0.00337902) 
 ("da1",1,0.00337019)       
 ("dey",5,0.00333783)       
 ("pérouse",2,0.00333376)   
 ("dilutes",3,0.00332366)   
 ("mennonites",3,0.00331573)
 ("eendracht",2,0.00330707) 
 ("brontë",3,0.0033024)     
 ("kudu",3,0.00329984)      

In [128]:
wv_a =  sum(all_word_sense_vectors(am, "bank").*disambiguate(am.vm, am.dict, w, l_use, false))
nearest_neighbors(am.vm, am.dict, wv_a)

10-element Array{Tuple{AbstractString,Int64,Float32},1}:
 ("doms",2,0.00349898)      
 ("identifed",2,0.00337904) 
 ("da1",1,0.00337019)       
 ("dey",5,0.00333786)       
 ("pérouse",2,0.00333378)   
 ("dilutes",3,0.00332369)   
 ("mennonites",3,0.00331575)
 ("eendracht",2,0.0033071)  
 ("brontë",3,0.0033024)     
 ("kudu",3,0.00329986)      

In [133]:
[wv wv_a wv-wv_a]

100×3 Array{Float32,2}:
 -0.000549951  -0.000549953   1.97906f-9 
  8.82424f-5    8.82341f-5    8.29459f-9 
 -0.000169456  -0.00016947    1.39116f-8 
  5.33938f-6    5.33941f-6   -2.91038f-11
  0.00181305    0.00181306   -1.22236f-8 
 -0.000852755  -0.000852769   1.45519f-8 
  0.000529032   0.000529015   1.70548f-8 
  0.000679199   0.000679156   4.26662f-8 
  0.000364573   0.000364558   1.57743f-8 
 -3.76887f-5   -3.76736f-5   -1.50612f-8 
 -0.000271979  -0.000271983   4.13274f-9 
 -0.000335945  -0.000335975   2.93076f-8 
  0.000768151   0.000768134   1.74041f-8 
 -0.000324189  -0.000324187  -1.97906f-9 
 -0.00020649   -0.000206471  -1.84955f-8 
 -0.000205744  -0.000205765   2.09693f-8 
 -0.000113083  -0.000113092   9.45874f-9 
  0.000643895   0.000643911  -1.60071f-8 
  8.59268f-5    8.59088f-5    1.80589f-8 
  6.41633f-5    6.41821f-5   -1.87574f-8 
  0.00128759    0.0012876    -1.00117f-8 
  0.00079151    0.000791473   3.6438f-8  
  0.000198559   0.00019857   -1.12341f-8 
 -0.000288

In [116]:
l_ee, l_ss, lem = lexically_informed_embeddings(db, am, "bank", "bank", 'n');

In [117]:
for (tle, ss) in zip(l_ee,l_ss)
    @show ss
    @show nearest_neighbors(am.vm, am.dict, tle)
    println("-"^32)
    println("\n")
end

ss = (n) bank building, bank (a building in which the business of banking transacted; "the bank is on the corner of Nassau and Witherspoon")
nearest_neighbors(am.vm,am.dict,tle) = Tuple{AbstractString,Int64,Float32}[("bank",5,1.95574),("banker",8,1.70124),("trust",12,1.67848),("savings",9,1.65595),("investments",5,1.6485),("stock",5,1.64087),("investor",3,1.63579),("real",13,1.63575),("j.p",3,1.63418),("investment",8,1.63387)]
--------------------------------


ss = (n) bank (a supply or stock held in reserve for future use (especially in emergencies))
nearest_neighbors(am.vm,am.dict,tle) = Tuple{AbstractString,Int64,Float32}[("bank",8,2.99978),("banks",1,2.85707),("deposits",2,2.84313),("bank's",1,2.83829),("exchange",2,2.8291),("balance",3,2.81137),("depositors",1,2.8109),("treasury",2,2.80644),("reserve",8,2.79827),("money",2,2.79216)]
--------------------------------


ss = (n) bank (sloping land (especially the slope beside a body of water); "they pulled the canoe up on the bank";

In [130]:
?AdaGram.var_update_z!

No documentation found.

`AdaGram.var_update_z!` is a `Function`.

```
# 2 methods for generic function "var_update_z!":
var_update_z!{Tw<:Integer}(vm::AdaGram.VectorModel, x::Tw, y::Tw, z::DenseArray{Float64,N<:Any}) at /home/ubuntu/.julia/v0.5/AdaGram/src/gradient.jl:93
var_update_z!{Tw<:Integer}(vm::AdaGram.VectorModel, x::Tw, y::Tw, z::DenseArray{Float64,N<:Any}, num_meanings::Int64) at /home/ubuntu/.julia/v0.5/AdaGram/src/gradient.jl:93
```


In [118]:
all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])
    

function _lexically_informed_embeddings(wn::DB,ee, word,lemma_word, pos)
    indexed_corpus = semcor #TODO: Pass this as a parameter, not as a gloel
    target_synset_examples = get_all_usages(wn, indexed_corpus, lemma_word, pos)
    target_synsets = collect(keys(target_synset_examples))
    lemma = db[lemma_word, pos]
        
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,(synset, examples)) in enumerate(target_synset_examples)
        embeddings[ii] = mean(examples) do eg#::AbstractVector
        context::Vector{SubString{String}} = filter(w::AbstractString->w!=word, eg)
        synthesize_embedding(ee,context, word)
        end
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sense_frequency(ss,lemma) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings(wn::DB, ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
        
_li_embeddings = Dict{Tuple{DB, Any, String, String, Char}, Tuple{Vector{Vector{Float32}}, Vector{Synset}, Lemma}}()
function lexically_informed_embeddings(wn::DB, ee, word,lemma_word, pos)
    get!(_li_embeddings, (wn, ee, word, lemma_word, pos)) do
        _lexically_informed_embeddings(wn, ee, word, lemma_word, pos)
    end
end

lexically_informed_embeddings (generic function with 2 methods)

In [119]:
function supervised_wsd(challenge, ee, wn::DB)
    try
        embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        #sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
        logprobs_of_context = [logprob_of_context(ee,challenge.context, wv; skip_oov=true, normalise_over_length=true) 
                                    for wv in embeddings] 
      
        sense_index= indmax(logprobs_of_context)

        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lemma)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB)
    try
        lemma = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lemma)
        
        sense_freqs =  Float32[sense_frequency(ss,lemma) for ss in target_synsets]      
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sensekey(wn, synset,  lemma)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 1 method)

In [120]:
using ProgressMeter

In [121]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    @showprogress for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB)
    method = challenge -> supervised_wsd(challenge, ee, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end


function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB)
    method = challenge -> most_frequent_sense(challenge, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [122]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db))

#println("only nouns  : \t\t", 
@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db)
#println("only verbs  : \t\t", 
@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db)
#println("only adjecti: \t\t", 
@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db)
#println("only adverbs: \t\t", 
@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:29
overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979

(0.7319587628865979,0.6826923076923077,0.7064676616915424)

In [123]:
?AdaGram.exp_normalize!

No documentation found.

`AdaGram.exp_normalize!` is a `Function`.

```
# 1 method for generic function "exp_normalize!":
exp_normalize!(x) at /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:120
```


# "models/adagram/more_senses.adagram_model.jld
### with Adagram Disambig weighting (prior used)
### with SemCor data 
overall: 		(0.655408489274304,0.6328779197884531,0.6439461883408072)
nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7161410018552876,0.6967509025270758,0.706312900274474)
vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5106382978723404,0.4873096446700508,0.4987012987012987)
aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6619718309859155,0.649171270718232,0.6555090655509065)
rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7268041237113402,0.6778846153846154,0.7014925373134329)

### with Adagram Disambig weighting (prior used)
### with just glosses 

overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)


In [168]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db))

Progress: 100%|█████████████████████████████████████████| Time: 0:04:42
overall: 		(0.6067567567567568,0.5936535918907008,0.6001336600579195)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
only nouns  : 		(0.6344383057090239,0.621841155234657,0.6280765724703737)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
only verbs  : 		(0.5129533678756477,0.5025380710659898,0.5076923076923077)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
only adjecti: 		(0.6275071633237822,0.6049723756906077,0.6160337552742615)
only adverbs: 		(0.6893203883495146,0.6826923076923077,0.6859903381642513)


In [19]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db))
println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db))
println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db))
println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db))


overall: 		(0.7783164389598942,0.7783164389598942,0.7783164389598942)
only nouns  : 		(0.7653429602888087,0.7653429602888087,0.7653429602888087)
only verbs  : 		(0.7529610829103215,0.7529610829103215,0.7529610829103215)
only adjecti: 		(0.8038674033149171,0.8038674033149171,0.8038674033149171)
only adverbs: 		(0.875,0.875,0.875)


In [21]:
#Zero shot WSI for 1 shot WSD
using Training

In [41]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=50)
rr.corpus_size = ee.corpus_size
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr);

In [42]:
rr.embedding["us"] |> length

50

In [43]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db))

overall: 		(0.6522522522522523,0.6381665932128691,0.6451325462241033)
only nouns  : 		(0.6869244935543278,0.6732851985559567,0.6800364630811304)
only verbs  : 		(0.5630397236614854,0.5516074450084603,0.5572649572649573)
only adjecti: 		(0.667621776504298,0.643646408839779,0.6554149085794655)
only adverbs: 		(0.6941747572815534,0.6875,0.6908212560386474)


In [44]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [45]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db))

overall: 		(0.654054054054054,0.6399294843543412,0.6469146803296949)
only nouns  : 		(0.6804788213627992,0.6669675090252708,0.6736554238833182)
only verbs  : 		(0.5544041450777202,0.5431472081218274,0.5487179487179487)
only adjecti: 		(0.6962750716332379,0.6712707182320442,0.6835443037974684)
only adverbs: 		(0.7233009708737864,0.7163461538461539,0.7198067632850241)


In [ ]:
using JLD